In [ ]:
from Bio import Entrez, SeqIO
import random
import os

In [ ]:
#dependency to check: NCBI Datasets CLI
#'conda install -c conda-forge ncbi-datasets-cli'

#if the download using CLI fails, you can use manual FTP download
#'wget -r -np -nH --cut-dirs=7 -R index.html* \
#ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/020/520/425/GCF_020520425.1/

In [2]:
#creating two variant parental genome VCF files
spinach_FASTA = "/spinach_genome/ncbi_dataset/data/GCF_020520425.1/GCF_020520425.1_BTI_SOV_V1_genomic.fna"